In [1]:
import json
import pandas as pd
import numpy as np

# Load JSON from file
with open("sample_data/demo_data.json", "r") as file:
    json_data = json.load(file)

In [2]:
# Flatten JSON
data = []
for product in json_data.get("demo_products", []):
    data.append({
        "id": product.get("id", np.nan),
        "name": product.get("name", None),
        "category": product.get("category", None),
        "price_amount": product.get("price", {}).get("amount", np.nan),
        "price_currency": product.get("price", {}).get("currency", None),
        "discount_percentage": product.get("price", {}).get("discount", {}).get("percentage", np.nan),
        "discount_valid_until": product.get("price", {}).get("discount", {}).get("valid_until", None),
        "in_stock": product.get("availability", {}).get("status", {}).get("in_stock", np.nan),
        "quantity": product.get("availability", {}).get("status", {}).get("quantity", np.nan)
    })


In [3]:

# Convert to a DataFrame for better visualization and further processing
df = pd.DataFrame(data)

# Replace NaN with None if needed
df = df.replace({np.nan: None})

# Display DataFrame
print(df)

# how to save into csv format
df.to_csv("sample_data/flattened_products.csv", index=False)

   id                           name     category price_amount price_currency  \
0   1  Wireless Bluetooth Headphones  Electronics        59.99            USD   
1   2                     Headphones  Electronics        59.99            USD   
2   3                            TVs  Electronics        59.99            USD   
3   4                        Laptops  Electronics        800.0            USD   

  discount_percentage discount_valid_until in_stock quantity  
0                10.0           2025-02-15     True    150.0  
1                None                 None     True    150.0  
2                10.0           2025-02-15     None     None  
3                10.0           2025-02-15     None     None  


# same logic as before in load to snoflake exerciee

In [4]:
import snowflake.connector
import os
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")


Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful


In [5]:
table_name = "DEMO_PRODUCTS"

# Create table if not exists
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INT,
    name STRING,
    category STRING,
    price_amount FLOAT,
    price_currency STRING,
    discount_percentage FLOAT,
    discount_valid_until DATE,
    in_stock BOOLEAN,
    quantity INT
);
"""
cursor.execute(create_table_query)
print(f"Table '{table_name}' ensured to exist.")

# Insert Data into Snowflake
for _, row in df.iterrows():
    insert_query = f"""
    INSERT INTO {table_name} (id, name, category, price_amount, price_currency, discount_percentage, discount_valid_until, in_stock, quantity)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, tuple(row))

print("Data inserted successfully!")

# Commit and close connection
session.commit()
cursor.close()
session.close()
print("Snowflake session closed.")

Table 'DEMO_PRODUCTS' ensured to exist.
Data inserted successfully!
Snowflake session closed.
